In [1]:
!pip install tensorflow matplotlib pillow

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.image import resize

In [3]:
# Function to load and preprocess image
def load_and_process_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return vgg19.preprocess_input(img)


In [4]:
# Function to deprocess image for display
def deprocess_image(x):
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    return np.clip(x[:, :, ::-1], 0, 255).astype('uint8')


In [ ]:
# Load images
content_path = tf.keras.utils.get_file("content.jpeg", "https://i.imgur.com/F28w3Ac.jpg")
style_path = tf.keras.utils.get_file("style.webp", "https://i.imgur.com/YoNe9Hk.jpg")

content_image = load_and_process_image(content_path)
style_image = load_and_process_image(style_path)

In [ ]:
# Define content and style layers
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']

In [ ]:
# Load VGG19 model for feature extraction
vgg = vgg19.VGG19(weights='imagenet', include_top=False)
vgg.trainable = False
outputs = [vgg.get_layer(name).output for name in style_layers + [content_layer]]
model = Model([vgg.input], outputs)

In [ ]:
# Compute feature representations
def get_features(image):
    preprocessed = tf.convert_to_tensor(image, dtype=tf.float32)
    return model(preprocessed)

In [ ]:
style_features = get_features(style_image)
content_features = get_features(content_image)

In [ ]:

# Extract style and content features separately
style_acts = style_features[:-1]
content_act = content_features[-1]


In [ ]:
# Gram matrix for style
def gram_matrix(tensor):
    x = tf.squeeze(tensor)
    x = tf.reshape(x, (-1, x.shape[-1]))
    return tf.matmul(x, x, transpose_a=True)

style_grams = [gram_matrix(f) for f in style_acts]


In [ ]:
# Initialize generated image
generated_image = tf.Variable(content_image, dtype=tf.float32)


In [ ]:
# Define loss and optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.02)

@tf.function
def compute_loss_and_grads(generated_image):
    with tf.GradientTape() as tape:
        features = get_features(generated_image)
        gen_style = features[:-1]
        gen_content = features[-1]

        style_loss = tf.add_n([
            tf.reduce_mean((gram_matrix(gen) - style_gram) ** 2)
            for gen, style_gram in zip(gen_style, style_grams)
        ]) / len(style_layers)

        content_loss = tf.reduce_mean((gen_content - content_act) ** 2)

        total_loss = 1e4 * style_loss + content_loss
    grads = tape.gradient(total_loss, generated_image)
    return total_loss, grads

In [ ]:
# Run optimization loop
epochs = 500
for i in range(epochs):
    loss, grads = compute_loss_and_grads(generated_image)
    optimizer.apply_gradients([(grads, generated_image)])
    if i % 100 == 0:
        print(f"Step {i}, Loss: {loss.numpy():.2f}")

In [ ]:
# Show final output
final_img = deprocess_image(generated_image.numpy())
plt.imshow(final_img)
plt.title("Stylized Image")
plt.axis("off")
plt.tight_layout()
plt.show()